In [25]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

In [26]:
pickle_file = '../input_data/notMnist/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 28, 28) (2000,)
Validation set (100, 28, 28) (100,)
Test set (100, 28, 28) (100,)


In [27]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

In [28]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 784) (2000, 10)
Validation set (100, 784) (100, 10)
Test set (100, 784) (100, 10)


In [29]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
          
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [30]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

In [31]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

In [32]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 13.050831
Training accuracy: 16.1%
Validation accuracy: 23.0%
Loss at step 100: 1.878719
Training accuracy: 74.3%
Validation accuracy: 68.0%
Loss at step 200: 1.304341
Training accuracy: 79.3%
Validation accuracy: 72.0%
Loss at step 300: 0.981615
Training accuracy: 82.2%
Validation accuracy: 72.0%
Loss at step 400: 0.770360
Training accuracy: 84.7%
Validation accuracy: 72.0%
Loss at step 500: 0.625875
Training accuracy: 86.6%
Validation accuracy: 72.0%
Loss at step 600: 0.522586
Training accuracy: 88.9%
Validation accuracy: 74.0%
Loss at step 700: 0.445865
Training accuracy: 90.7%
Validation accuracy: 72.0%
Loss at step 800: 0.386200
Training accuracy: 92.5%
Validation accuracy: 72.0%
Test accuracy: 70.0%


In [33]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # Assignment 3 : Add l2 regularization to the loss...
    # Restrict the weights
    loss = tf.reduce_mean(loss + 0.01 * tf.nn.l2_loss(weights))
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" %
                  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.841618
Minibatch accuracy: 7.0%
Validation accuracy: 16.0%
Minibatch loss at step 500: 0.720338
Minibatch accuracy: 90.6%
Validation accuracy: 74.0%
Minibatch loss at step 1000: 0.598308
Minibatch accuracy: 86.7%
Validation accuracy: 73.0%
Minibatch loss at step 1500: 0.710082
Minibatch accuracy: 80.5%
Validation accuracy: 73.0%
Minibatch loss at step 2000: 0.521260
Minibatch accuracy: 89.8%
Validation accuracy: 76.0%
Minibatch loss at step 2500: 0.531264
Minibatch accuracy: 89.1%
Validation accuracy: 72.0%
Minibatch loss at step 3000: 0.522726
Minibatch accuracy: 89.8%
Validation accuracy: 77.0%
Test accuracy: 79.0%


In [35]:
num_nodes= 1024
batch_size = 128

tf.reset_default_graph()

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset= tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases = tf.Variable(tf.zeros([num_nodes]))
    
    weights_1 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_1 = tf.Variable(tf.zeros([num_labels]))
    
    relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    
    logits = tf.matmul(relu_layer, weights_1) + biases_1
    
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    # Assignment number 3    
    regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_1)
    loss = tf.reduce_mean(loss + 0.01 * regularizer)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction_1 = tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_1) + biases_1)
    test_prediction_1 =  tf.nn.softmax(
     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_1) + biases_1)

In [36]:
steps = 3001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" %
                  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction_1.eval(), valid_labels))
            print("============================")
            print("Test accuracy: %.1f%%" % accuracy(test_prediction_1.eval(), test_labels))

Minibatch loss at step 0: 3464.188721
Minibatch accuracy: 7.0%
Validation accuracy: 29.0%
Test accuracy: 31.0%
Minibatch loss at step 500: 21.149706
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Test accuracy: 80.0%
Minibatch loss at step 1000: 0.680280
Minibatch accuracy: 97.7%
Validation accuracy: 77.0%
Test accuracy: 84.0%
Minibatch loss at step 1500: 0.566923
Minibatch accuracy: 96.9%
Validation accuracy: 77.0%
Test accuracy: 83.0%
Minibatch loss at step 2000: 0.502925
Minibatch accuracy: 97.7%
Validation accuracy: 76.0%
Test accuracy: 79.0%
Minibatch loss at step 2500: 0.475365
Minibatch accuracy: 98.4%
Validation accuracy: 79.0%
Test accuracy: 82.0%
Minibatch loss at step 3000: 0.455955
Minibatch accuracy: 98.4%
Validation accuracy: 78.0%
Test accuracy: 80.0%
